# Estimating Spectra

trying different estimators for spectra

In [ ]:
import os
import sys
sys.path.append("..")
import dask
import scipy as sp
import numpy as np
import mtspec
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
# matplotlib.rc_file('../rc_file_paper')
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
# sys.path.append("..")
# from paths import path_samoc, path_results
import statsmodels.tsa as tsa
from statsmodels.tsa.arima_process import ArmaProcess
# from bb_analysis_timeseries import AnalyzeTimeSeries as ATS

In [ ]:
hg = xr.open_dataarray('../../data/AMO_climexp/iamo_hadsst_gmst.nc' , decode_times=False)  # months since 1850-01-15
eg = xr.open_dataarray('../../data/AMO_climexp/iamo_ersst_gmst.nc'  , decode_times=False)  # months since 1880-01-15
hs = xr.open_dataarray('../../data/AMO_climexp/iamo_hadsst_gmsst.nc', decode_times=False)  # months since 1853-01-15
es = xr.open_dataarray('../../data/AMO_climexp/iamo_ersst_gmsst.nc' , decode_times=False)  # months since 1854-01-15
thg, teg, ths, tes = 1850, 1880, 1853, 1854

In [ ]:
ds = xr.open_dataset('../../results/all_timeseries.nc', decode_times=False)
tf = ds['AMO_had']
ttf = 1870

In [ ]:
tf.dropna(dim='time')

In [ ]:
f, ax = plt.subplots(5,1, figsize=(12,12), constrained_layout=True)
for i, (ts, t) in enumerate(zip([hg,eg,hs,es,tf],[thg, teg, ths, tes, ttf])):
    # time series
    ax[0].plot(ts.time/12+t, ts.values+i, label=['hg','eg','hs','es', 'tf'][i])
    
    # periodogram for common period 1880-2019
    ts_ = ts[12*(1880-t):12*(2019-t)].fillna(0)
    print(np.isnan(ts_.values).sum())
    freq, Pxx = periodogram(ts_)
    ax[1].loglog(freq, Pxx, alpha=.3)
    
    # Welch periodogram Hanning window
    freq, Pxx = sp.signal.welch(ts_, window='bartlett', nperseg=len(ts_))
    ax[2].loglog(freq, Pxx)
    
    # Lomb-Scargle periodogram for full time series
    freq = np.linspace(0.0005,1/2,150)
    lsp = sp.signal.lombscargle(ts.time.where(ts.notnull(), drop=True).values, 
                                ts.where(ts.notnull(), drop=True).values, freq)
    ax[3].loglog(freq, lsp)
    
    # Welch periodogram Hanning window
    freq, Pxx = sp.signal.welch(ts_, window='hann', nperseg=59*12)
    ax[4].loglog(freq, Pxx)
print(1/freq[1]/12)
for i in range(4):
    ax[i+1].set_xlim((5e-4,.5))
    ax[i+1].set_ylabel(['periodogram', 'tapered data periodogram', 'Lomb-Scargle', 'Welch'][i])
ax[0].legend()
ax[-1].set_xlabel(r'frequency [month$^{-1}$]')

### AMO
- `iamo_e4rsst_gmst.nc`: AMO index SST 25-60N, 7-75W minus regression on global mean temperature, as in van Oldenborgh et al 2009, based on ERSST v5 from NCDC, AMO [C] SST,
- `iamo_hadsst_gmsst.nc`: AMO index SST EQ-60N, 0-80W minus SST 60S-60N, as in Trenberth and Shea 2006, based on HadSST 4.0.0.0,

### PDO
- `ipdo` PDO [1] Pacific Decadal Oscillation index, from SWFSC
- `ipdo_hadsst3`  patternfield: projecting variable eof1, month 1 of pattern Monthly 5 degree version of HadSST.3.1.1.0 - Median of realisations on field Monthly 5 degree version of HadSST.3.1.1.0 - Median of realisations
- `ipdo_ersst` patternfield: projecting variable eof1, month 1 of pattern NOAA ERSSTv5 (in situ only) on field NOAA ERSSTv5 (in situ only),

## Why is the periodogram is a bad estimator?
1. not robust: more data points do not lead to a decrease in the estimate variance
1. spectral leakage: intensitiesare contaminated with variance from far aways frequencies. Can be solved with tapers.

I will use three time series with known spectra to illustrate the problems and compare different spectral estimators.
1. `A` white noise time series: theoretical power spectrum $\Gamma(\omega) = \mathrm{var}(A)$
1. `B` AR(2) process ($\alpha_1=0.3, \alpha_2=0.3$)
1. `C` AR(2) process ($\alpha_1=0.9, \alpha_2=-0.8$)



Ghil et al. (2000): 
> For instrumental climate records, with a typical length of a few hundred points, the choice p ⫽ 2 and K ⫽ 3 offers a good compromise between the required frequency resolution for resolving distinct cli- mate signals (e.g., ENSO and decadal-scale variability) and the benefit of multiple spectral degrees of freedom, i.e., of reduced variance [e.g., Mann and Park, 1993].

## Autoregressive processes

The order $p$ autoregressive process is defined as
$$ X_t = \sum_{p=1}^{p} \phi_p X_{t-p} + \sigma_\epsilon^2 \epsilon_t + c$$
where $\epsilon_t$ is a white noise process.

The mean of an AR(1) process is
$$\mu_X = \frac{c}{1-\phi_1}$$
while the variance is 
$$\mathrm{var}(X(t)) = \frac{\sigma_\epsilon^2}{1-\phi_1^2}$$

Yule-Walker method for estimating the autocorrelation and noise amplitude for an AR(1) process:

\begin{align}
    \hat{\phi}_1 & = r_1 \\
    \hat{\sigma}_\epsilon^2 & = T^{-1} \sum_{t=1}^T (X'_t - \hat{\phi}_1 X'_t)^2
\end{align}
where $X'_t = X_t - \bar{X}$

The theoretical power spectrum of an AR(2) process is (p. 224 von Storch and Zwiers)
$$\Gamma(\omega) = \frac{\sigma_\epsilon^2}{1 + \alpha_1^2 + \alpha_2^2 - 2\alpha_1 (1 - \alpha_2) \cos(2 \pi \omega) + 2 \alpha_2 \cos(4 \pi \omega)}$$


#### MTM-SVD spectra of M. Mann

http://www.meteo.psu.edu/holocene/public_html/Mann/tools/tools.html

In [ ]:
for N in [100,150,250,1000]:
    a_ = np.array([.1*np.sin(2*np.pi/82*t) + .05*np.sin(2*np.pi/23*t+.5) for t in np.arange(12*N)/12])
    b_ = a_ + ArmaProcess(np.array([1, -.8]), np.array([1])).generate_sample(scale=.2, nsample=12*N)# np.random.rand((12*N))
    b_ -= np.mean(b_)
    # data -= np.mean(data)
    a = xr.DataArray(data=a_, dims='time', coords={'time':np.arange(12*N)/12})
    b = xr.DataArray(data=b_, dims='time', coords={'time':np.arange(12*N)/12})
    # a -= xr_lintrend(a)
    
    f, ax = plt.subplots(3,1, figsize=(6.4,5))
    
    ax[0].plot(a.time, a, alpha=.7)
    ax[0].plot(b.time, b, alpha=.7)
    
    for i, da in enumerate([a,b]):
        ax[i+1].axvline(1/82, c='k')
        ax[i+1].axvline(1/23, c='k')
        
        freq, spec = ATS(da).mtspectrum()
        ax[i+1].plot(freq*12, spec, ls='--', c='C0')

        freq, spec = ATS(da).periodogram()
        ax[i+1].plot(freq*12, spec, ls='-', c='C1')

        freq, spec = ATS(da).Welch()
        ax[i+1].plot(freq*12, spec, ls='-.', c='C2')

        freq, spec = ATS(da).mtspectrum(tb=2, nt=5)
        ax[i+1].plot(freq*12, spec, ls=':', c='C3')

        ax[i+1].loglog()
    ax[1].set_ylim(1e-3,1e1)   
    ax[2].set_ylim(1e-1,1e2)

In [ ]:
def periodogram(ts, d=1.):
    """ naive absolute squared Fourier components 
    
    input:
    ts .. time series
    d  .. sampling period
    """
    freq = np.fft.rfftfreq(len(ts))
    Pxx = 2*np.abs(np.fft.rfft(ts))**2/len(ts)
    return freq, Pxx

def Welch(ts, d=1, window='hann'):
    """ Welch spectrum """
    return sp.signal.welch(ts, window=window)

def mtspectrum(ts, d=1., tb=4, nt=4):
    """ multi-taper spectrum 
    
    input:
    ts .. time series
    d  .. sampling period
    tb .. time bounds (bandwidth)
    nt .. number of tapers
    """
    spec, freq, jackknife, _, _ = mtspec.mtspec(
                data=ts, delta=d, time_bandwidth=tb,
                number_of_tapers=nt, statistics=True)
    return freq, spec

In [ ]:
def AR2_process(a1, a2, N):
    """ generates AR(2) time series of length N """
    return ArmaProcess(np.array([1, -a1, -a2]), np.array([1])).generate_sample(nsample=N)

def AR2_spectrum(sz, a1, a2, omega):
    """ theoretical AR(2) spectrum 
    
    input:
    sz     .. standard deviation
    a1, a2 .. AR(2) coefficients
    omega  .. frequency at which to evaluate expression
    """
    # multiplied here by an extra factor of 2, so that we can integrate over [0,0.5] and not over [-0.5.0.5]
    return 2*sz**2/(1+a1**2+a2**2-2*(a1*(1-a2)*np.cos(2*np.pi*omega)+a2*np.cos(4*np.pi*omega)))

N = 2000
A = np.random.rand((N))-.5
A /= np.std(A)
b = 0.3
B_ = AR2_process(b, b, N)
B = B_ + 10*np.sin(2*np.pi*.3162*np.arange((N))+.5763)
# B /= np.std(B)
c1, c2 = .9, -.8
C = AR2_process(c1, c2, N)

In [ ]:
f, ax = plt.subplots(4,3,figsize=(12,10), constrained_layout=True)
for i, ts in enumerate([A,B,C]):
    ax[0,i].axhline(0, c='grey', lw=.5)
    ax[0,i].plot(np.arange(240), ts[:240])
    
    for j, ts_ in enumerate([ts[:120], ts[:240], ts]):
        ts_ -= np.mean(ts_)
        ax[j+1,i].plot([0,0],[0,1], c='w', label=f'var(x)={np.var(ts_):7.3f}')
        freq = np.linspace(0,.5,len(ts_))
        if i==0:  
            ax[j+1,i].axhline(2*np.var(ts_), c='r', label=f'theor. {np.var(ts_):7.3f}')
            # multiplied here by an extra factor of 2, so that we can integrate over [0,0.5] and not over [-0.5.0.5]
        if i==1:
            spec = AR2_spectrum(sz=1, a1=b , a2=b , omega=freq)
            ax[j+1,i].plot(freq, spec, c='r', label=f'theor. {np.sum((freq[1]-freq[0])*spec)+50:7.3f}')
            # variance of sin with amplitude A: A^2/2
            ax[j+1,i].axvline(.3162, c='r')

        if i==2:
            spec = AR2_spectrum(sz=1 , a1=c1, a2=c2, omega=freq)
            ax[j+1,i].plot(freq, 10*np.log10(spec), c='r', label=f'theor. {np.sum((freq[1]-freq[0])*spec):7.3f}')
            ax[j+1,i].axhline(0, c='r')
            
        freq, Pxx = periodogram(ts_)
        label = f'periodogram {np.sum((freq[1]-freq[0])*Pxx):7.3f} $\sigma_S=${np.std(Pxx):7.3f}'
        if i<2:  ax[j+1,i].bar(freq[1:], Pxx[1:]             , width=.001, alpha=.5, label=label)
        else  :  ax[j+1,i].bar(freq[1:], 10*np.log10(Pxx[1:]), width=.003, alpha=.5, label=label)
        
        for window in ['hann', 'boxcar', 'bartlett']:
            f, Pxx = sp.signal.welch(ts, window=window, nperseg=60)#, average='median')
            label = f'Welch {window} {np.sum((f[1]-f[0])*Pxx):7.3f}'
            if i<2:  ax[j+1,i].scatter(f[1:], Pxx[1:]             , s=8, label=label)
            else  :  ax[j+1,i].scatter(f[1:], 10*np.log10(Pxx[1:]), s=8, label=label)        
        
        freq, Pxx = mtspectrum(ts_)
        label= f'MT spec {np.sum((freq[1]-freq[0])*Pxx):7.3f} $\sigma_S=${np.std(Pxx):7.3f}'
        if i<2:  ax[j+1,i].scatter(freq[1:], Pxx[1:]             , s=10, marker='s', label=label)
        else  :  ax[j+1,i].scatter(freq[1:], 10*np.log10(Pxx[1:]), s=10, marker='s', label=label)
        
        if i==0: ax[j+1,i].set_ylim((0,8))
        if i==1: ax[j+1,i].set_ylim((0,20))
        ax[j+1,i].legend(fontsize=8)
        
for i in range(3):
    ax[0,i].set_title(['white noise', r'AR(2) (0.3,0.3) + $10 \sin(2 \pi 0.3162 t + 0.5763)$','AR(2) (0.9,-0.8)'][i])
    ax[i,0].set_ylabel([r'time series $x(t)$', 'n=120\npower spectral density','n=240\npower spectral density'][i])
    ax[-1,i].set_xlabel(r'frequency [$\Delta t^{-1}$]')

## index time series

In [ ]:
plt.plot(xr.open_dataset(f'{path_samoc}/SST/PMV_EOF_20N_had.nc').pcs.isel(mode=0))
(ds['TPI_had']/ds['TPI_had'].std()).plot()

In [ ]:
# das = []
# for idx in ['AMO', 'TPI', 'SOM']:
#     for run in ['had', 'ctrl', 'lpd']:
#         if run=='ctrl':   ts, Nt = '_51_301', 250*12
#         elif run=='lpd':  ts, Nt = '_154_404', 250*12
#         elif run=='had':  ts, Nt = '', 149*12
#         fn = f'{path_samoc}/SST/{idx}_ds_dt_raw_{run}{ts}.nc'
#         print(os.path.exists(fn))
#         da = xr.open_dataarray(fn, decode_times=False).assign_coords(time=np.arange(Nt))
#         da.name = f'{idx}_{run}'
#         das.append(da)

In [ ]:
ds = xr.open_dataset(f'{path_results}/all_timeseries.nc')

In [ ]:
fig, ax = plt.subplots(4,3,figsize=(12,15),constrained_layout=True, sharey='row')
for i, idx in enumerate(['AMO', 'TPI', 'SOM']):
    for j, run in enumerate(['had', 'ctrl', 'lpd']):
        c = f'C{i}'
        key = f'{idx}_{run}'
        x = ds[key].dropna(dim='time')
        f = np.linspace(0,0.5,len(x))
        std_ = ds[key].std().values
        AM = ARMA(endog=x.values, order=(1,0)).fit()
        ac, std = AM.arparams[0], np.sqrt(AM.sigma2)
        ax[0,j].plot(ds.time/12+[1870,51,154][j], ds[key]-i, label=fr'{idx} $\phi_1=${ac:.2f}, $\sigma_x=${std_:.2f}, $\sigma_\epsilon=${std:.2f}', c=c)
        ax[0,j].axhline(-i, c=c, lw=.5)
        
        for k in range(3):  ax[k+1,j].plot(f[1:]*12, AR2_spectrum(sz=std, a1=ac, a2=0, omega=f)[1:]*10**(-[4,1.5,2][k]*i), c=c, ls='-.')
        
        f, Pxx = periodogram(x)
        ax[1,j].plot(f[1:]*12, Pxx[1:]*10**(-4*i), c=c, alpha=.5, label=fr'{idx}')#' $\rho_1$={ac:.3f}')
        mc = mc_ar1_spectrum(x.values, spectrum=periodogram)
        ax[1,j].plot(mc['freq'][1:]*12, mc['median'][1:]*10**(-4*i), c=c, ls='-' , label='median')
        ax[1,j].fill_between(mc['freq'][1:]*12, mc['1'][1:]*10**(-4*i), mc['99'][1:]*10**(-4*i), color=c, alpha=0.2)
        ax[1,j].fill_between(mc['freq'][1:]*12, mc['5'][1:]*10**(-4*i), mc['95'][1:]*10**(-4*i), color=c, alpha=0.2)
        
        f, Pxx = Welch(x)
        ax[2,j].plot(f[1:]*12, Pxx[1:]*10**(-1.5*i), c=c, alpha=.5, label=fr'{idx} $\rho_1$={ac:.3f}')
        
        f, Pxx = mtspectrum(x)
        ax[3,j].plot(f[1:]*12, Pxx[1:]*10**(-2*i), c=c, alpha=.5, label=fr'{idx} $\rho_1$={ac:.3f}')
        mc = mc_ar1_spectrum(x.values, spectrum=mtspectrum)
        ax[3,j].plot(mc['freq'][1:]*12, mc['median'][1:]*10**(-2*i), c=c, ls='-' , label='median')
        ax[3,j].fill_between(mc['freq'][1:]*12, mc['1'][1:]*10**(-2*i), mc['99'][1:]*10**(-2*i), color=c, alpha=0.2)
        ax[3,j].fill_between(mc['freq'][1:]*12, mc['5'][1:]*10**(-2*i), mc['95'][1:]*10**(-2*i), color=c, alpha=0.2)
        
for i in range(3):
    ax[0,i].set_title(['HIST', 'HIGH', 'LOW'][i])
    ax[0,i].set_xlabel(r'time [month]')
    ax[-1,i].set_xlabel(r'frequency [year$^{-1}$]')
    ax[1,i].set_ylim((1e-13, 1e2))
    ax[0,i].legend(fontsize=8)
    ax[1,i].legend(fontsize=7, ncol=3)
    for j in range(3):
        ax[j+1,i].set_xlim((12*3e-4, 12*6e-1))
        ax[j+1,i].set_xscale('log', basex=10)
        ax[j+1,i].set_yscale('log', basey=10)

for i in range(4):  ax[i,0].set_ylabel(['index anomaly [K]', r'periodogram [K$^2$ month]', r'Welch [K$^2$ month]', r'Multi-taper [K$^2$ month]'][i])
plt.savefig(f'{path_results}/spectra/indices, spectra')

## Fitting AR(1) spectra: theoretical vs. MC spectral estimates

In [ ]:
from statsmodels.tsa.arima_model import ARMA

def mc_ar1_ARMA(phi, std, n, N=1000):
    """ Monte-Carlo AR(1) processes
    
    input:
    phi .. (estimated) lag-1 autocorrelation
    std .. (estimated) standard deviation of noise
    n   .. length of original time series
    N   .. number of MC simulations 
    """
    AR_object = ArmaProcess(np.array([1, -phi]), np.array([1]), nobs=n)
    mc = AR_object.generate_sample(nsample=(N,n), scale=std, axis=1, burnin=1000)
    return mc

def mc_ar1_spectrum(x, spectrum, N=1000, filter_type=None, filter_cutoff=None):
    """ calculates the Monte-Carlo spectrum and the 95% confidence interval
    
    input:
    x             .. time series
    spectrum      .. spectral density estimation function
    N             .. number of MC simulations
    filter_type   ..
    filter_cutoff ..
    
    output:
    mc_spectrum   .. 
    """
    AM = ARMA(endog=x, order=(1,0)).fit()
    phi, std = AM.arparams[0], np.sqrt(AM.sigma2)
    mc = mc_ar1_ARMA(phi=phi, std=std, n=len(x), N=N)
    mc_spectra = np.zeros((N, int(len(mc[0,:])/2)+1))
    for i in range(N):  freq, mc_spectra[i,:] = spectrum(mc[i,:])
    mc_spectrum = {}
    mc_spectrum['median'] = np.median(mc_spectra, axis=0)
    mc_spectrum['freq'] = freq
    for p in [1,2.5,5,95,97.5,99]:
        mc_spectrum[str(p)] = np.percentile(mc_spectra, p, axis=0)
    return mc_spectrum

## testing AR(1) spectral density estimates: theoretical vs. Monte-Carlo

In [ ]:
# long AR(1) processes and their MC spectra
f, ax = plt.subplots(3,1, figsize=(8,12))
for i, std in enumerate([.1, .5, .9]):
    n, phi = 10000, 0.5
    x = ArmaProcess(np.array([1, -phi]), np.array([1])).generate_sample(nsample=n, scale=std, burnin=1000)

    f, Pxx = mtspectrum(x)
    ax[i].plot(f, Pxx, alpha=.5)
    Pxx_th = AR2_spectrum(sz=std, a1=ac, a2=0, omega=f)
    ax[i].plot(f[1:], Pxx_th[1:], c='C3')

    mc_ = mc_ar1_spectrum(std=std, phi=phi, n=n, spectrum=mtspectrum)
    ax[i].plot(mc_[1,1:], mc_[0,1:], c='C1')
    ax[i].plot(mc_[1,1:], mc_[2,1:], c='C1')
    ax[i].plot(mc_[1,1:], mc_[3,1:], c='C1')

    AM = ARMA(endog=x, order=(1,0)).fit()
    ac, std = AM.arparams[0], np.sqrt(AM.sigma2)
    mc = mc_ar1_spectrum(std=std, phi=ac, n=len(x), spectrum=mtspectrum)
    ax[i].plot(mc[1,1:], mc[0,1:], c='C2', ls=':' , alpha=.8)
    ax[i].plot(mc[1,1:], mc[2,1:], c='C2', ls='--', alpha=.8)
    ax[i].plot(mc[1,1:], mc[3,1:], c='C2', ls='--', alpha=.8)
    ax[i].plot(mc[1,1:], mc[4,1:], c='C2', ls='--', alpha=.8)
    ax[i].plot(mc[1,1:], mc[5,1:], c='C2', ls='--', alpha=.8)

    ax[i].plot(f[1:], AR2_spectrum(sz=std, a1=ac, a2=0, omega=f)[1:], c='C3')

    ax[i].set_yscale('log', basey=10)

    print(f'{np.var(x):7.3f}, {np.sum((f[1]-f[0])*Pxx):7.3f}, {np.sum((f[1]-f[0])*Pxx_th):7.3f}, {np.sum((mc_[1,1]-mc_[1,0])*mc_[0,:]):7.3f}, {np.sum((mc[1,1]-mc[1,0])*mc[0,:]):7.3f}')

In [ ]:
for n in [10, 100, 1000, 10000]:
    for phi in [.1,.5,.9]:
        for std in [.1,.5,.9]:
            x = ArmaProcess(np.array([1, -phi]), np.array([1])).generate_sample(nsample=n, scale=std, burnin=1000)
            # Yule Walker estimate 
            phi_YW = np.corrcoef(x[:-1], x[1:])[1,0]
            y = x-np.mean(x)
            std_YW = np.sqrt(np.sum((y[1:]-phi_YW*y[:-1])**2)/(len(y)-1))
            # max likelihood estimation of parameters
            AM = ARMA(endog=x, order=(1,0)).fit()
            phi_ML, std_ML = AM.arparams[0], np.sqrt(AM.sigma2)
            print(f'{n:5}: phi={phi:3.1f}, std={std:3.1f},   YW: {phi_YW:6.3f}, {std_YW:6.3f}, MLE: {phi_ML:6.3f}, {std_ML:6.3f}')


In [ ]:
fig, ax = plt.subplots(2,3,figsize=(12,6),constrained_layout=True, sharey='row')
for i, idx in enumerate(['AMO', 'TPI', 'SOM']):
    for j, run in enumerate(['had', 'ctrl', 'lpd']):
        c = f'C{i}'
        key = f'{idx}_{run}'
        AM = tsa.arima_model.ARMA(endog=ds[key].dropna(dim='time').values, order=(1,0)).fit()
        ac, std = AM.arparams[0], np.sqrt(AM.sigma2)
#         std = ds[key].std().values
#         ac = np.corrcoef(ds[key].dropna(dim='time').values[1:], ds[key].dropna(dim='time').shift(time=1).dropna(dim='time').values)[1,0]
        ax[0,j].plot(ds.time, ds[key]+i, label=fr'{idx} $\sigma=${std:.2f}', c=c)
        
        f, Pxx = mtspectrum(ds[key].dropna(dim='time').values)
        ax[1,j].plot(f[1:], Pxx[1:]*10**(2*i), c=c, alpha=.5, label=fr'{idx} $\rho_1$={ac:.3f}')
        ax[1,j].plot(f[1:], AR2_spectrum(sz=std, a1=ac, a2=0, omega=f)[1:]*10**(2*i), c=c)
        
        mc = mc_ar1_spectrum(std=std, phi=ac, n=len(ds[key].dropna(dim='time')), spectrum=mtspectrum)
        ax[1,j].plot(mc[1,1:], mc[0,1:]*10**(2*i), c=c, ls='-')
        ax[1,j].plot(mc[1,1:], mc[2,1:]*10**(2*i), c=c, ls='--')
        ax[1,j].plot(mc[1,1:], mc[3,1:]*10**(2*i), c=c, ls='--')
        ax[1,j].plot(mc[1,1:], mc[4,1:]*10**(2*i), c=c, ls=':')
        ax[1,j].plot(mc[1,1:], mc[5,1:]*10**(2*i), c=c, ls=':')
        

for i in range(3):
    ax[0,i].set_title(['HIST', 'HIGH', 'LOW'][i])
    ax[0,i].legend(fontsize=10)
    ax[1,i].legend(fontsize=10)
#     for j in range(3):
    ax[1,i].set_xlim((3e-4, 6e-1))
    ax[1,i].set_xscale('log', basex=10)
    ax[1,i].set_yscale('log', basey=10)

for i in range(2):  ax[i,0].set_ylabel(['time series', 'Multi-taper'][i])
    
# ax2.fill_between(freq, jackknife[:, 0], jackknife[:, 1], color="red", alpha=0.2)

In [ ]:
ac = .9
print(f'X_t       {0/(1-ac):6.3f}      {1/(1-ac**2):7.4f}')
for n in [10,100,1000,10000,100000,1000000]:
    a = ArmaProcess(np.array([1, -ac]), np.array([1])).generate_sample(nsample=n, burnin=1000)
    
    print(f'{n:7d}:  {np.mean(a):6.3f}      {np.var(a):7.4f}')
    

## testing $\int \hat{S}(f) \, \mathrm{d}f = \mathrm{var}(x(t))$

In [ ]:
def int_Sf(f, Pxx):
    df = f[1]-f[0]
    return np.sum(df*Pxx)

print('\033[1midx run   var.  perio Welch MTM   theor  MCper MCmtm    \033[0m')
for i, idx in enumerate(['AMO', 'TPI', 'SOM']):
    for j, run in enumerate(['had', 'ctrl', 'lpd']):
        c = f'C{i}'
        key = f'{idx}_{run}'
#         ac = np.corrcoef(ds[key].dropna(dim='time').values[1:], ds[key].dropna(dim='time').shift(time=1).dropna(dim='time').values)[1,0]
        x = ds[key].dropna(dim='time')
        var = ds[key].var().values
        AM = tsa.arima_model.ARMA(endog=ds[key].dropna(dim='time').values, order=(1,0)).fit()
        ac, std = AM.arparams[0], np.sqrt(AM.sigma2)
        f = np.linspace(0,.5,len(x))
        p = int_Sf(*periodogram(x))
        w = int_Sf(*Welch(x))
        m = int_Sf(*mtspectrum(x))
        t = int_Sf(f, AR2_spectrum(sz=std, a1=ac, a2=0, omega=f))
        c = int_Sf(f, mc_ar1_spectrum(x.values, spectrum=periodogram)['median'])
        d = int_Sf(f, mc_ar1_spectrum(x.values, spectrum=mtspectrum)['median'])
        print(f'{idx} {run:4} {var:.3f}  {p:.3f} {w:.3f} {m:.3f} {t:.3f}  {c:.3f} {d:.3f}    {var/c:7.3f} {var/d:7.3f}    {c/var:7.3f} {d/var:7.3f}')
    print()

- the equality holds for the periodogram, the Welch method (with some small error), and the multi-taper method
- theoretical and Monte-Carlo median spectra do not work correctly though!

In [ ]:
# theoretical vs. Monte-Carlo
n = 1000
f = np.linspace(0,.5,n)
df = f[1]-f[0]
fig, ax = plt.subplots(3,3, figsize=(8,5), sharey=True, sharex=True, constrained_layout=True)
for i, std in enumerate([0.1,.5,1]):
    for j, ac in enumerate([.1,.5,.9]):
        ax[i,j].plot([0,0],[0,1], c='w', label=fr'var$(X_t) = {std**2/(1-ac**2):.4f}$')

        spec = AR2_spectrum(sz=std, a1=ac, a2=0, omega=f)
        ax[i,j].plot(f, spec, label=fr'theor. $\int S = {np.sum(df*spec):.4f}$')
        
        mc_spec = mc_ar1_spectrum(phi=ac, std=std, n=n, spectrum=periodogram)
        ax[i,j].plot(mc_spec[1,:], mc_spec[0,:], c='C1', label=fr'median  $\int S = {np.sum(df*mc_spec[0,:]):.4f}$')
        ax[i,j].plot(mc_spec[1,:], mc_spec[2,:], c='C1', ls='--')  # 5%
        ax[i,j].plot(mc_spec[1,:], mc_spec[3,:], c='C1', ls='--')  # 95%
        
        
        ax[i,j].set_xscale('log', basex=10)
        ax[i,j].set_yscale('log', basey=10)
        ax[i,j].set_ylim((1e-4, 1e3))
        
        if i==0:  ax[i,j].set_title(fr'$\rho_1=${ac}', fontsize=16)
        if j==0:  ax[i,j].set_ylabel(fr'$\sigma=${std}', fontsize=16)
        ax[i,j].legend(fontsize=8)

In [ ]:
from mtspec import mtspec
from mtspec.util import _load_mtdata

data = _load_mtdata('v22_174_series.dat.gz')

# Calculate the spectral estimation.
spec, freq, jackknife, _, _ = mtspec(
    data=data, delta=4930.0, time_bandwidth=3.5,
    number_of_tapers=5, nfft=312, statistics=True)

fig = plt.figure()
ax1 = fig.add_subplot(2, 1, 1)
# Plot in thousands of years.
ax1.plot(np.arange(len(data)) * 4.930, data, color='black')
ax1.set_xlim(0, 800)
ax1.set_ylim(-1.0, 1.0)
ax1.set_xlabel("Time [1000 years]")
ax1.set_ylabel("Change in $\delta^{18}O$")

ax2 = fig.add_subplot(2, 1, 2)
ax2.set_yscale('log')
# Convert frequency to Ma.
freq *= 1E6
ax2.plot(freq, spec, color='black')
ax2.fill_between(freq, jackknife[:, 0], jackknife[:, 1],
                 color="red", alpha=0.3)
ax2.set_xlim(freq[0], freq[-1])
ax2.set_ylim(0.1E1, 1E5)
ax2.set_xlabel("Frequency [c Ma$^{-1}]$")
ax2.set_ylabel("Power Spectral Density ($\delta^{18}O/ca^{-1}$)")

plt.tight_layout()


In [ ]:
len(data)

In [ ]:
len(spec)

In [ ]:
from pandas.tools.plotting import autocorrelation_plot

In [ ]:
autocorrelation_plot(AMO_ctrl)
autocorrelation_plot(AMO_ctrl)